In [51]:
import pandas as pd
import pickle
import numpy as np
import re 
import datetime as dt

In [52]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [53]:
pd.set_option('display.max_colwidth', None)

In [54]:
# Read both datasets
os_kb_entities=pd.read_csv(f'open_sanctions_entities.csv',index_col=0)
os_kb_entities['kb_origin']='open_sanctions'
ls_kb_entities=pd.read_csv(f'lilsis_entities.csv',index_col=0)
ls_kb_entities['kb_origin']='lilsis'

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [56]:
# Rename columns to concat datasets  
os_kb_entities=os_kb_entities.rename(columns={'full_notes':'desc'})
ls_kb_entities=ls_kb_entities.rename(columns={'primary_ext':'schema','end_date':'deathdate','aliases':'AKA','start_date':'birthdate','context':'desc'})

In [57]:
# Concat datasets into one
kb_entities=pd.concat([os_kb_entities,ls_kb_entities]).reset_index().rename(columns={'index':'original_index'})

In [58]:
# Drop useless columns
kb_entities.drop(['schema','notes','context','types','start_date_sentence','end_date_sentence'],1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [59]:
kb_entities.shape

(503310, 11)

In [60]:
# Fix trailing whitespaces
kb_entities['desc']=kb_entities['desc'].apply(lambda x: re.sub(r"\b(\.)[\.\s]+$", "\\1", x))

In [61]:
# Remove entities with no description
kb_entities=kb_entities[kb_entities['desc'].str.replace(' ','').apply(len)>0]

In [62]:
# Drop duplicated based on same name and description
kb_entities.drop_duplicates(subset=['name','desc'],inplace=True)

In [63]:
kb_entities.shape

(429953, 11)

In [64]:
## Resolve duplicate entitiy IDs

In [65]:
# Find duplicate entries on the 'id' columns
redundant_entities_by_id=kb_entities[kb_entities['id'].duplicated(keep=False)].sort_values(['id','name'])

In [66]:
# Drop duplicate_entities (these were all cases where the ID was taken from Wikidata)
redundant_entities_indices=redundant_entities_by_id.index
kb_entities.drop(redundant_entities_indices, inplace=True)

In [67]:
# Keep first of duplicated entities
redundant_entities_by_id_consolidated_desc=redundant_entities_by_id.groupby(['id']).first().reset_index()

In [68]:
# Concatenate back to kb entity dataframe 
kb_entities=pd.concat([kb_entities,redundant_entities_by_id_consolidated_desc])
kb_entities.reset_index(drop=True,inplace=True)

In [69]:
kb_entities.shape

(429861, 11)

In [70]:
## Resolve duplicates on birthdate, deathdate and website

In [71]:
# Create auxiliary column for ordering based on len of desc field
kb_entities['desc_len']=kb_entities['desc'].str.len()

In [72]:
redundancy_cols=['birthdate','deathdate','website']
for col in redundancy_cols:
    # Find duplicates ordered by description len
    redundant_entities_by_col=kb_entities[~(kb_entities[col].isna())&
                (kb_entities.duplicated(['name',col],keep=False))
               ].sort_values(by=['name','desc_len'],ascending=False)

    # Drop duplicate_entities on name and birthdate
    redundant_entities_indices=redundant_entities_by_col.index
    kb_entities.drop(redundant_entities_indices, inplace=True)

    # Keep first of duplicated entities
    redundant_entities_by_col_consolidated_desc=redundant_entities_by_col.groupby(['name',col]).first().reset_index()

    # Concatenate back to kb entity dataframe 
    kb_entities=pd.concat([kb_entities,redundant_entities_by_col_consolidated_desc])
    kb_entities.reset_index(drop=True,inplace=True)

In [73]:
kb_entities.shape

(428519, 12)

In [74]:
## Further duplicate inspection 

In [75]:
# Duplicates on name
kb_duplicated=kb_entities[kb_entities.duplicated(keep=False,subset=['name'])].sort_values(by=['name'])

In [76]:
kb_entities.shape

(428519, 12)

In [78]:
# Standardise names 
kb_entities['name']=kb_entities['name'].str.title()

In [79]:
# Ensure person name is included in all descriptions
name_not_in_notes_indices=kb_entities[kb_entities.apply(lambda x: x['name'].lower() not in x['desc'].lower(),axis=1)].index.values
naming_string = 'This person is called '
kb_entities.loc[name_not_in_notes_indices,'desc'] = kb_entities.loc[name_not_in_notes_indices].apply(lambda x: naming_string + x['name'] + '. ' + x['desc'],axis=1)

In [80]:
# Ensure descriptions end in stop mark. 
kb_entities.loc[~kb_entities['desc'].isna(),'desc']=kb_entities.loc[~kb_entities['desc'].isna(),'desc'].apply(lambda x: x + '.' if x[-1]!='.' else x)

In [81]:
# Clean up data by removing multiple trailing stop marks
multi_stopmarks_expr = re.compile('\.\s?\.')

for expr in [multi_stopmarks_expr]:
    # Replace expression in string
    kb_entities.loc[~kb_entities['desc'].isna(),'desc']=kb_entities.loc[~kb_entities['desc'].isna(),'desc'].apply(lambda x: ''.join([re.sub(expr, '. ', x)]))

In [83]:
## Add KB URLs

In [84]:
def url_generator(id_,name, dataset):
    if dataset=='open_sanctions':
        return f'https://www.opensanctions.org/entities/{id_}'
    if dataset=='lilsis':
        return f'https://littlesis.org/person/{id_}-{name}'

In [85]:
kb_entities['kb_url']=kb_entities.apply(
    lambda x: [url_generator(x['id'],x['name'], 'open_sanctions') 
                             if x['kb_origin']=='open_sanctions' 
                             else url_generator(x['id'],x['name'], 'lilsis') 
                                                  ][0],1
)

In [86]:
dataset='full'
kb_iteration=dt.datetime.now().strftime('%Y_%m_%d')
dataset=f'{dataset}_{kb_iteration}'
kb_entities.to_csv(f'kb_entities_{dataset}.csv')